# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

Variables of note:

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
'1000000' = 1.000000

FHINCP - income flag - 1: yes https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

GRNTP - gross rent (monthly) (not 0: n/a,not paying rent, owned)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GRNTP.json

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#pumas that extend outside of the city '0400112','0400129'

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP'

In [4]:
#AMI bands in 2021 by household size
#represents household income limit by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71000,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
#Housing costs affordable to different AMI bands based on household size
#assumes maxium of 30% of annual household income, translated to monthly rent
unit_afford = {'1':[0,415,691,1106,1383,1000000000],\
              '2':[0,475,790,1265,1580,1000000000],\
              '3':[0,549,888,1423,1775,1000000000],\
              '4':[0,663,988,1580,1975,1000000000],\
              '5':[0,776,1068,1708,2135,1000000000],\
              '6':[0,890,1146,1834,2293,1000000000],\
              '7':[0,1003,1225,1960,2450,1000000000],\
              '8':[0,1117,1304,2450,2608,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [9]:
df.head()

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,WGTP2,WGTP3,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
4,0400119,-60000.0,1.0,0.0,1.029928,1000000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,0400129,-60000.0,1.0,0.0,1.029928,1000000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0400120,-60000.0,1.0,0.0,1.029928,1000000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,0400116,-60000.0,1.0,0.0,1.029928,1000000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,0400129,-60000.0,1.0,0.0,1.029928,1000000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df.shape

(7489, 87)

In [11]:
df.to_excel('output/raw_Census_rent.xlsx',index=False)

In [12]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)
df['HINCP'] = df.ADJINC * df.HINCP

In [13]:
dfff = df[(df.HHSz.isna())&(df.HINCP==-60000)|(df.HINCP==0)&(df.GRNTP==0)].copy()

In [14]:
dfff.WGTP.sum()

4045.0

In [15]:
dfff.head()

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,WGTP2,WGTP3,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz
3897,0400129,0.0,2.0,73.0,1.029928,1000000.0,0.0,71.0,126.0,117.0,...,21.0,76.0,20.0,78.0,22.0,60.0,70.0,71.0,115.0,2
5017,0400123,0.0,1.0,43.0,1.029928,1000000.0,0.0,11.0,11.0,76.0,...,16.0,43.0,13.0,84.0,44.0,12.0,11.0,40.0,40.0,1
5641,0400121,0.0,5.0,110.0,1.029928,1000000.0,0.0,35.0,102.0,111.0,...,106.0,177.0,113.0,104.0,206.0,106.0,29.0,35.0,30.0,5
7277,0400112,0.0,2.0,48.0,1.029928,1000000.0,0.0,83.0,83.0,50.0,...,49.0,15.0,15.0,83.0,82.0,14.0,47.0,80.0,49.0,2
8202,0400121,0.0,1.0,117.0,1.029928,1000000.0,0.0,116.0,213.0,124.0,...,131.0,114.0,131.0,121.0,113.0,208.0,175.0,198.0,209.0,1


In [16]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)&(df.HINCP!=0)&(df.GRNTP!=0)].copy()

In [17]:
dff['hou_cost'] = dff.GRNTP
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [18]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

In [19]:
dff['cost_burdened'] = np.where(dff.hou_cost>dff.aff_cost,'burdened','not burdened')

In [20]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [21]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,WGTP2,WGTP3,...,WGTP77,WGTP78,WGTP79,WGTP80,HHSz,hou_cost,aff_cost,AMI_range,cost_burdened,unit_aff
3612,0400120,177353.6016,3.0,42.0,1.029928,1000000.0,2460.0,74.0,13.0,71.0,...,42.0,44.0,43.0,42.0,3,2460.0,4921.562444,o100_ami,not burdened,o100_ami
3637,0400119,55307.1336,7.0,172.0,1.029928,1000000.0,1405.0,70.0,81.0,157.0,...,152.0,164.0,282.0,65.0,7,1405.0,1534.772957,50_80_ami,not burdened,50_80_ami
3639,0400116,70344.0824,2.0,177.0,1.029928,1000000.0,1808.0,244.0,245.0,293.0,...,173.0,299.0,161.0,194.0,2,1808.0,1952.048287,o100_ami,not burdened,o100_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [22]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [23]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','GRNTP','hou_cost', 'aff_cost','HHSz']

In [24]:
table = dff.copy().drop(columns=drop_cols)

In [25]:
table = table.groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()

In [26]:
table_2 = table.copy().groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [27]:
rent_ami = table.copy().drop(columns=['unit_aff','cost_burdened']).groupby(['GEO_ID','AMI_range']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)

In [28]:
rent_aff = table.copy().drop(columns=['AMI_range','cost_burdened']).groupby(['GEO_ID','unit_aff']).sum().reset_index()
rent_aff = make_est(rent_aff)
rent_aff = rent_aff.drop(columns=repwts)

In [29]:
cost_burd = table.copy().drop(columns=['unit_aff','AMI_range']).groupby(['GEO_ID','cost_burdened']).sum().reset_index()
cost_burd = make_est(cost_burd)
cost_burd = cost_burd.drop(columns=repwts)

In [30]:
cost_burd_ami = table.copy().drop(columns=['unit_aff']).groupby(['GEO_ID','cost_burdened','AMI_range']).sum().reset_index()
cost_burd_ami = make_est(cost_burd_ami)
cost_burd_ami = cost_burd_ami.drop(columns=repwts)

In [31]:
table_3 = table.copy().drop(columns='cost_burdened').groupby(['GEO_ID','AMI_range','unit_aff']).sum().reset_index()
table_3 = make_est(table_3)
table_3 = table_3.drop(columns=repwts)

In [32]:
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff','cost_burdened'],aggfunc=np.sum).reset_index()

rent_ami = pd.pivot_table(rent_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range'],aggfunc=np.sum).reset_index()

rent_aff = pd.pivot_table(rent_aff,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

cost_burd = pd.pivot_table(cost_burd,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['cost_burdened'],aggfunc=np.sum).reset_index()

cost_burd_ami = pd.pivot_table(cost_burd_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['cost_burdened','AMI_range'],aggfunc=np.sum).reset_index()

In [33]:
table_3 = pd.pivot_table(table_3,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff'],aggfunc=np.sum).reset_index()

In [34]:
with pd.ExcelWriter(f'output/pums_grossrent_gap_june.xlsx') as writer:
    table_2.to_excel(writer,sheet_name='full_renter')
    rent_ami.to_excel(writer,sheet_name='hh_ami_lvl_renter')
    rent_aff.to_excel(writer,sheet_name='hh_afford_byAMI_renter')
    cost_burd.to_excel(writer,sheet_name='cost_burdened_renter')
    cost_burd_ami.to_excel(writer,sheet_name='cost_burdened_byAMI_renter')
    table_3.to_excel(writer,sheet_name='demand_supply_amilvl')